https://docs.api.lens.org/request.html

Get token: https://www.lens.org/lens/user/subscriptions

In [47]:
import getpass
import pandas as pd
import requests
import sys
import time

In [2]:
token=getpass.getpass('token:')

token:········


In [3]:
url = 'https://api.lens.org/scholarly/search'
size=1000

In [4]:
def query(affiliation="University of Antioquia",range_type="date_published",
          date_ini="2003-01-01",
          date_end="2004-12-31",size=1000):
    if range_type=="year_published":
        date_ini=date_ini.split('-')[0]
        date_end=date_end.split('-')[0]
    data='''{
    "query": {
       "bool": {
         "must": [{
           "match_phrase":{
                    "author.affiliation.name": "%s"
               }
           },
           {
           "range": {
           "%s": {
                    "gte": "%s",
                    "lte": "%s"
                }
           } 
           }
         ]
       }
     },
     "size": %i
    }''' %(affiliation,range_type,date_ini,date_end,size)
    return data

In [5]:
data=query(affiliation="University of Antioquia",range_type="date_published",
      date_ini="2003-01-01",date_end="2004-12-31",size=1000)

In [13]:
data=query(affiliation="University of Antioquia",range_type="year_published",
      date_ini="2003",date_end="2004",size=1000)

In [16]:
headers = {'Authorization': '{}'.format(token), 'Content-Type': 'application/json'}
response = requests.post(url, data=data, headers=headers)
if response.status_code != requests.codes.ok:
  print(response.status_code)
else:
  r=response.text

In [17]:
rr=pd.read_json(r)

In [18]:
rr.shape

(823, 3)

In [21]:
def get_query(token,data,url='https://api.lens.org/scholarly/search',size=1000):
    headers = {'Authorization': '{}'.format(token), 'Content-Type': 'application/json'}
    response = requests.post(url, data=data, headers=headers)
    if response.status_code != requests.codes.ok:
        sys.exit(response.status_code)
    else:
        return response.text    

In [9]:
udea=pd.DataFrame( list( rr.data.values ) )

In [300]:
year_ranges=[["1900","1994"],["1995","2000"],["2001","2003"]   ]

In [312]:
udea=pd.DataFrame()
for i in range(len(year_ranges)):
    print(year_ranges[i])
    data=query(affiliation="University of Antioquia",range_type="year_published",
      date_ini=year_ranges[i][0],date_end=year_ranges[i][1],size=1000)
    r=get_query(token,data)
    print("query:",pd.read_json(r).shape[0] )
    udea=udea.append( pd.read_json(r)  ).reset_index(drop=True)
    tmp=udea.copy()
    time.sleep(2)

['1900', '1994']
query: 926
['1995', '2000']
query: 943
['2001', '2003']
query: 971


In [309]:
date_ranges=[["2004-01-01","2005-11-30"],["2005-12-01","2006-12-31"],["2007-01-01","2007-12-31"],
             ["2008-01-01","2008-10-31"],["2008-11-01","2009-05-31"],['2009-06-01','2009-12-31'],
             ['2010-07-01', '2010-12-31'],['2011-01-01', '2011-05-31'],['2011-06-01', '2011-12-31'],
             ['2012-01-01', '2012-05-31'],['2012-06-01', '2012-11-30'],['2012-12-01', '2013-03-31'],
             ['2013-04-01', '2013-10-31'],['2013-11-01', '2014-02-28'],['2014-03-01', '2014-10-31'],
             ['2014-11-01', '2015-02-28'],['2015-03-01', '2015-10-31'],['2015-11-01', '2016-03-31'],
             ['2016-04-01', '2016-09-30'],['2016-10-01', '2016-12-31'],['2017-01-01', '2017-06-30'],
             ['2017-07-01', '2017-12-31'],['2018-01-01', '2018-08-31'],['2018-09-01', '2019-06-30']
            ]

In [303]:
FIND_DATES=False
if FIND_DATES:
    m=["","01-31","02-28","03-31","04-30","05-31","06-30","07-31","08-31","09-30","10-31","11-30","12-31"]
    dr=date_ranges[0][1].split('-')

    year="2019-"
    month=6
    if int(dr[1])<12:
        date_ranges=[[dr[0]+'-'+m[int(dr[1])+1].split('-')[0]+'-'+'01',year+m[month]]]
    else:
        date_ranges=[[str(int(dr[0])+1)+'-'+'01'+'-'+'01',year+m[month]]]
    print( date_ranges )

In [315]:
for i in range(6,7):print(i)

6


In [322]:
rini=6
for i in range(22,len(date_ranges)):
    print(date_ranges[i])
    data=query(affiliation="University of Antioquia",range_type="date_published",
      date_ini=date_ranges[i][0],date_end=date_ranges[i][1],size=1000)
    r=get_query(token,data)
    kk=pd.read_json(r)
    print("query:",i,kk.shape[0] )
    if FIND_DATES:
        if kk.shape[0]>=1000 and month>1:
            month=month-1
            date_ranges=[[date_ranges[0][0],year+m[month]]]
        
    udea=udea.append( kk  ).reset_index(drop=True)
    tmp=udea.copy()

    time.sleep(5)

['2018-01-01', '2018-08-31']
query: 22 971
['2018-09-01', '2019-06-30']
query: 23 841


In [327]:
udea=pd.DataFrame( list( udea.data.values ) ).reset_index(drop=True)

In [328]:
udea.shape

(23830, 32)

In [329]:
udea.to_json('data/udea.json.gz',compression='gzip')

In [330]:
pd.read_json('data/udea.json.gz',compression='gzip').reset_index(drop=True)

,abstract,author_count,authors,chemicals,clinical_trials,conference,created,date_published,end_page,external_ids,...,references,references_count,scholarly_citations,scholarly_citations_count,source,source_urls,start_page,title,volume,year_published
0,"H a sta h ace unos pocos anos, el carcinoma de...",4,"[{'collective_name': None, 'last_name': 'Uribe...",None,None,None,2018-05-16T03:24:40.711000+00:00,1981-01-01T00:00:00+00:00,140,"[{'value': '2286199698', 'type': 'magid'}]",...,None,NaN,None,NaN,None,"[{'type': 'html', 'url': 'https://revista.feco...",136,Ca. De endometrio en pacientes jovenes,32,1981
1,The chronic financial crisis of the Colombia e...,1,"[{'collective_name': None, 'last_name': 'Franc...",None,None,None,2018-05-15T16:38:38.283000+00:00,1986-01-01T00:00:00+00:00,96,"[{'value': '2088684644', 'type': 'magid'}]",...,None,NaN,[178-901-825-050-525],1.0,{'title': 'Lecturas de Economía'},"[{'type': 'html', 'url': 'http://aprendeenline...",33,Incidencia de los problemas financieros en los...,19,1986
2,None,1,"[{'collective_name': None, 'last_name': 'Ochoa...",None,None,None,2018-05-12T23:59:40.672000+00:00,None,583,"[{'value': '10.1016/s0022-5347(17)36659-4', 't...",...,None,NaN,"[018-754-522-850-518, 033-592-588-139-053, 036...",4.0,"{'asjc_codes': ['2748'], 'asjc_subjects': ['Ur...",None,580,The Urofacial (Ochoa) Syndrome Revisited,148,1992
3,Una de las conclusiones finales del articulo '...,1,"[{'collective_name': None, 'last_name': 'Arena...",None,None,None,2018-05-16T06:15:16.604000+00:00,1985-01-01T00:00:00+00:00,60,"[{'value': '2337241128', 'type': 'magid'}]",...,None,NaN,None,NaN,None,"[{'type': 'html', 'url': 'https://aprendeenlin...",51,INFLUENCIA DEL CONTROL INTERNO EN LA AUDITORÍA,None,1985
4,None,1,"[{'collective_name': None, 'last_name': 'Echev...",None,None,None,2018-05-14T21:26:15.931000+00:00,1999-01-01T00:00:00+00:00,49,"[{'value': '1498919540', 'type': 'magid'}]",...,None,NaN,None,NaN,None,"[{'type': 'html', 'url': 'https://biblat.unam....",41,El umbral de la muerte,None,1999
5,None,5,"[{'collective_name': None, 'last_name': 'Posad...",None,None,None,2018-05-16T19:41:59.744000+00:00,2011-12-01T00:00:00+00:00,622,"[{'value': '2619256199', 'type': 'magid'}]",...,None,NaN,None,NaN,{'title': 'Revista Colombiana De Ciencias Pecu...,None,617,Exigencias energéticas para ganado de carne,24,2011
6,None,2,"[{'collective_name': None, 'last_name': 'Ferná...",None,None,None,2018-05-14T14:11:03.232000+00:00,2011-07-01T00:00:00+00:00,111,"[{'value': '165265220', 'type': 'magid'}]",...,None,NaN,None,NaN,None,None,104,As habilidades metacognitivas na escritura dig...,8,2011
7,None,5,"[{'collective_name': None, 'last_name': 'Puert...",None,None,None,2018-05-14T23:56:53.871000+00:00,2011-06-01T00:00:00+00:00,172,"[{'value': '1597620011', 'type': 'magid'}]",...,None,NaN,None,NaN,None,"[{'type': 'html', 'url': 'http://www.scielo.or...",164,Early childhood caries in 1-5 year-old childre...,22,2011
8,Este articulo discute dos posturas teoricas so...,1,"[{'collective_name': None, 'last_name': 'Casta...",None,None,None,2018-05-16T02:04:46.362000+00:00,2011-06-01T00:00:00+00:00,86,"[{'value': '2263818659', 'type': 'magid'}]",...,"[{'lens_id': '002-623-390-958-55X'}, {'lens_id...",24.0,None,NaN,None,"[{'type': 'html', 'url': 'http://pepsic.bvsalu...",73,Social Intelligence and its Implications for t...,3,2011
9,El estudio de las unidades neologicas de una l...,2,"[{'collective_name': None, 'last_name': 'Berri...",None,None,None,2018-05-14T17:35:38.483000+00:00,2011-10-24T00:00:00+00:00,None,"[{'value': '588595799', 'type': 'magid'}]",...,"[{'lens_id': '033-162-367-944-34X'}, {'lens_id...",5.0,None,NaN,{'title': 'Debate Terminológico'},"[{'type': 'html', 'url': 'https://seer.ufrgs.b...",None,La neología en el ámbito de la música: la form...,None,2011
